In [0]:
%pip install --upgrade -Uqqq mlflow>=3.0 xgboost optuna uv

from pipeline import run_pipeline

import mlflow

mlflow.set_registry_uri("databricks-uc")
#model, metrics = run_pipeline(spark)

In [0]:
os.environ["DATABRICKS_HOST"]

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
    url = 'https://dbc-3e741eea-9ae2.cloud.databricks.com/serving-endpoints/up_down_forecaster/invocations'
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

In [0]:
import pandas as pd
import numpy as np

dataframe = pd.DataFrame(np.random.randint(0, 1, (4, 9)))
dataframe.columns = ['lag_0', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8']

In [0]:
score_model(dataframe)